In [21]:
COMPUTE_CV = True
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

import sys
sys.path.append("../input/utils")
import helpers
# DATA_PATH = "../input/shopee-product-matching/"
DATA_PATH = "../input/"


train = pd.read_csv(DATA_PATH + "train.csv")
train["target"] = train.label_group.map(train.groupby("label_group").posting_id.agg("unique").to_dict())
train["target"] = train["target"].apply(lambda x: " ".join(x))
train["image"] = DATA_PATH + "train_images/" + train["image"]
ttrain=train
if COMPUTE_CV == False:
    train = pd.read_csv(DATA_PATH + "test.csv")
    train["image"] = DATA_PATH + "test_images/" + train["image"]

train.head()

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,../input/train_images/0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,train_129225211 train_2278313361
1,train_3386243561,../input/train_images/00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,train_3386243561 train_3423213080
2,train_2288590299,../input/train_images/000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,train_2288590299 train_3803689425
3,train_2406599165,../input/train_images/00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,4093212188,train_2406599165 train_3342059966
4,train_3369186413,../input/train_images/00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,train_3369186413 train_921438619


In [13]:
import eca_nfnet_l1_arc_face

img_embs = eca_nfnet_l1_arc_face.get_test_embeddings(
    train
)

Building Model Backbone for eca_nfnet_l1 model


  0%|          | 0/4282 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Our image embeddings shape is torch.Size([34250, 11014])


In [14]:
import xlm_v1_arcface

bert_embs = xlm_v1_arcface.get_test_embeddings(
    train,"title"
)

Building Model Backbone for ../input/shopee-models/paraphrase-xlm-r-multilingual-v1 model


get_bert_embeddings: 100%|██████████████████| 1072/1072 [00:19<00:00, 56.30it/s]


In [15]:
print(type(img_embs),type(bert_embs))
print(img_embs.shape,bert_embs.shape)

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([34250, 11014]) torch.Size([34250, 768])


In [16]:
np.save("../input/img_embs.npy", img_embs.cpu())
np.save("../input/bert_embs.npy", bert_embs.cpu())

In [17]:
img_embs=np.load("../input/img_embs.npy")
bert_embs=np.load("../input/bert_embs.npy")
img_embs = torch.from_numpy(img_embs)
bert_embs = torch.from_numpy(bert_embs)

In [18]:

img_embs=F.normalize(img_embs)
bert_embs=F.normalize(bert_embs)

set_size = len(img_embs)
print(set_size)


test_df = helpers.add_measurements(train)

new_embs = helpers.blend_embs([img_embs, bert_embs], test_df)

combined_inds, combined_dists = helpers.combined_distances(new_embs) #n*K, 返回第i个样本第j大的相似度和对应的索引
# helpers.check_measurements(combined_dists, combined_inds, test_df)
pairs = helpers.sorted_pairs(combined_dists, combined_inds) #对矩阵


34250


In [24]:
import matplotlib.pyplot as plt
# ds=[[],[]]
# la=[]
# for x,y,v in pairs:
#     ds[0].append(min(1,np.dot(img_embs[x].cpu(), img_embs[y].cpu())))
#     ds[1].append(min(1,np.dot(bert_embs[x].cpu(), bert_embs[y].cpu())))
#     # print(x)
#     la.append(ttrain.iloc[x].label_group==ttrain.iloc[y].label_group)
    # print(x,y,v)
# print(mi)
# print(ds[0])
# plt.scatter(ds[0],ds[1], c=la)
# plt.xlim([0, 1.1])
# plt.ylim([0, 1.1])
# plt.show()


1746750


In [22]:
groups = [[] for _ in range(set_size)]
groups_p = [[] for _ in range(set_size)]
for x,y,v in pairs:
    groups[x].append(y)
    groups_p[x].append(v)
for pos, size_pct in helpers.get_targets_shape(ttrain):
    helpers.chisel(groups, groups_p, pos, int(size_pct * len(groups))) # 一组的长度，该长度的组数
    # print(pos, int(size_pct * len(groups)))

In [23]:
matches = [' '.join(test_df.iloc[g].posting_id.to_list()) for g in groups]
test_df['matches'] = matches

test_df[['posting_id','matches']].to_csv('submission.csv',index=False)
pd.read_csv('submission.csv').head()

,posting_id,matches
0,train_129225211,train_2278313361 train_129225211
1,train_3386243561,train_3386243561 train_3423213080
2,train_2288590299,train_2288590299 train_3803689425
3,train_2406599165,train_2406599165 train_1744956981 train_1508100548 train_3526771004
4,train_3369186413,train_921438619 train_3369186413


In [ ]:
# print(train['target'].head(), test_df['matches'].head())
# import eval_preds
# train['precision'],train['recall'],train['f1'] =  eval_preds.get_score(train['target'], test_df['matches'])
# print(train['f1'].mean(),train['recall'].mean(),train['precision'].mean())